**Data Preprocessing**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/README.txt~
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/README.txt
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/._README.txt
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/UCSDped1.m
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/._UCSDped1.m
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/UCSDped1.m~
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test001/079.tif
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test001/084.tif
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test001/123.tif
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test001/130.tif
/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomal

**Model Selection and Training** 
Convolutional Autoencoder (CAE)

imports of libraries

In [2]:
import os
import numpy as np
import cv2  # For image processing
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import img_to_array, load_img


**Data Preprocessing**

setting the path to the dataset

In [3]:
import os

# Set the path to your dataset on Kaggle
# This is a generic placeholder, replace 'dataset-directory-name' with your actual dataset directory name as found on Kaggle
dataset_path = '/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2'

# List files in the dataset directory
files = os.listdir(dataset_path)
print("Files in dataset directory:", files)

Files in dataset directory: ['UCSDped1', 'README.txt~', 'README.txt', '._README.txt', 'UCSDped2']


paths to specific folders within the UCSD Anomaly Detection Dataset

In [4]:
train_path_ped1 = '/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train'
train_path_ped2 = '/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Train'
test_path_ped1 = '/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'
test_path_ped2 = '/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Test'


**load and preprocess images**

In [ ]:
def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    for root, _, files in os.walk(folder):  # os.walk allows recursive search
        for filename in files:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                img = load_img(file_path, target_size=target_size, color_mode="grayscale")
                img = img_to_array(img) / 255.0
                images.append(img)
    return np.array(images)
#testing my function
images = load_images_from_folder(train_path_ped1)
print(images.shape)


In [ ]:
# Load images from both folders
train_images_ped1 = load_images_from_folder(train_path_ped1)
train_images_ped2 = load_images_from_folder(train_path_ped2)

# Print the shapes of the loaded image arrays
print(f"Shape of images from UCSDped1: {train_images_ped1.shape}")
print(f"Shape of images from UCSDped2: {train_images_ped2.shape}")

# Concatenate the images from both folders
train_images = np.concatenate([train_images_ped1, train_images_ped2])

# Print the shape of the concatenated image array
print(f"Shape of concatenated image array: {train_images.shape}")  # Should show (num_images, 128, 128, 1)


**Convolutional Autoencoder (CAE)**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# Define the autoencoder model with additional layers and a larger latent space
input_img = Input(shape=(128, 128, 1))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(256, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
# Model definition
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

**training the autoencoder**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback to monitor validation loss and stop training if it stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the autoencoder on a small subset of data first to check if it works
sample_train_images = train_images[:100]  # Taking a smaller subset of data for initial testing

autoencoder.fit(
    sample_train_images, sample_train_images,  # Using the same images for autoencoder
    epochs=10,  # Reduce epochs for testing purposes
    batch_size=16,  # Reduced batch size to avoid memory overload
    shuffle=True,
    validation_split=0.1,  # Use 10% of the data for validation
    callbacks=[early_stopping]  # Add early stopping callback
)


**Data Preprocessing**

In [ ]:
from tensorflow.keras.utils import img_to_array, load_img
from PIL import UnidentifiedImageError
import os
import numpy as np

def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    for root, _, files in os.walk(folder):  # Recursively search all subdirectories
        for filename in files:
            if filename.lower().endswith(valid_extensions):
                file_path = os.path.join(root, filename)
                print(f"Attempting to load: {file_path}")
                try:
                    # Load the image and convert to grayscale if necessary
                    img = load_img(file_path, target_size=target_size, color_mode="grayscale")
                    img = img_to_array(img) / 255.0  # Normalize the image
                    images.append(img)
                except (UnidentifiedImageError, OSError) as e:
                    print(f"Skipping file {file_path}: {e}")
    return np.array(images)


In [ ]:
test_images_ped1 = load_images_from_folder(test_path_ped1)
test_images_ped2 = load_images_from_folder(test_path_ped2)
test_images = np.concatenate([test_images_ped1, test_images_ped2])

print("Shape of test images:", test_images.shape)


In [ ]:
# Train the autoencoder (increase epochs as needed)
epochs = 100  # Try increasing to 100 or more if you have the computational resources
history = autoencoder.fit(
    test_images, test_images,
    epochs=epochs,
    batch_size=64,
    shuffle=True,
    validation_split=0.1
)

# Generate decoded images
decoded_imgs = autoencoder.predict(test_images)

# Display original and reconstructed images
import matplotlib.pyplot as plt

n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original images
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_images[i].reshape(128, 128), cmap='gray')
    plt.title("Original")
    plt.axis('off')
    
    # Display reconstructed images
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(128, 128), cmap='gray')
    plt.title("Reconstructed")
    plt.axis('off')

plt.show()

In [ ]:
import numpy as np

# Mean Squared Error (MSE) per image
reconstruction_errors = np.mean((test_images - decoded_imgs) ** 2, axis=(1, 2, 3))

# Calculate the average MSE across all images
average_mse = np.mean(reconstruction_errors)
print(f"Average MSE for all images: {average_mse}")

# Set a threshold for anomalies (e.g., 95th percentile)
anomaly_threshold = np.percentile(reconstruction_errors, 95)
anomalies = test_images[reconstruction_errors > anomaly_threshold]

print(f"Number of anomalies detected: {len(anomalies)}")


In [ ]:
import numpy as np

# Mean Squared Error (MSE) per image
reconstruction_errors = np.mean((test_images - decoded_imgs) ** 2, axis=(1, 2, 3))

# Set a threshold for anomalies (e.g., 95th percentile)
anomaly_threshold = np.percentile(reconstruction_errors, 90)
anomalies = test_images[reconstruction_errors > anomaly_threshold]

# Filter reconstruction errors for anomalies only
anomaly_errors = reconstruction_errors[reconstruction_errors > anomaly_threshold]

# Calculate the average MSE for images classified as anomalies
average_anomaly_mse = np.mean(anomaly_errors)
print(f"Average MSE for anomalous images: {average_anomaly_mse}")

# Number of anomalies detected
print(f"Number of anomalies detected: {len(anomalies)}")


In [ ]:
plt.figure(figsize=(20, 4))
for i in range(min(10, len(anomalies))):  # Show up to 10 anomalies
    ax = plt.subplot(1, 10, i + 1)
    plt.imshow(anomalies[i].reshape(128, 128), cmap='gray')
    plt.title("Anomaly")
    plt.axis('off')
plt.show()


In [ ]:
import numpy as np

# Assume reconstruction_errors contains the reconstruction error for each image in the test set
# Set a threshold for anomaly detection (e.g., 95th percentile)
threshold = np.percentile(reconstruction_errors, 95)

# Predict labels based on the threshold
predicted_labels = (reconstruction_errors > threshold).astype(int)  # 1 for anomaly, 0 for normal

# If you know the dataset is mostly normal, you can estimate an "accuracy" by assuming normal images are correctly classified.
# Accuracy estimation: proportion of normal images correctly identified as normal
normal_accuracy = np.mean(predicted_labels == 0)
print(f"Estimated accuracy for detecting normal images: {normal_accuracy:.2f}")


In [ ]:
import matplotlib.pyplot as plt

plt.hist(reconstruction_errors, bins=50)
plt.axvline(x=threshold, color='red', linestyle='--', label='Anomaly Threshold')
plt.xlabel("Reconstruction Error")
plt.ylabel("Frequency")
plt.legend()
plt.title("Distribution of Reconstruction Errors")
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def highlight_anomalies_on_original(original, reconstructed, error_threshold=0.1, radius=10):
    # Calculate absolute difference between original and reconstructed images
    error_map = np.abs(original - reconstructed)
    
    # Threshold the error map to isolate high-error regions
    anomaly_mask = (error_map > error_threshold).astype(np.uint8)
    
    # Convert to a format suitable for contour detection
    anomaly_mask = (anomaly_mask * 255).astype(np.uint8)

    # Find contours in the thresholded mask
    contours, _ = cv2.findContours(anomaly_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Convert original image to RGB to draw colored circles
    highlighted_image = cv2.cvtColor((original * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)

    # Draw circles around each contour directly on the original image
    for contour in contours:
        (x, y), contour_radius = cv2.minEnclosingCircle(contour)
        if contour_radius > radius:
            center = (int(x), int(y))
            cv2.circle(highlighted_image, center, int(contour_radius), (255, 0, 0), 2)  # Red circle

    return highlighted_image

# Set the number of images to display in the grid
n_images = 10  # Display the top 10 images with high reconstruction errors
sorted_indices = np.argsort(reconstruction_errors)[::-1]  # Descending order

plt.figure(figsize=(15, 10))
for i in range(n_images):
    idx = sorted_indices[i]
    
    # Original and reconstructed images
    original = test_images[idx].reshape(128, 128)
    reconstructed = decoded_imgs[idx].reshape(128, 128)
    highlighted_image = highlight_anomalies_on_original(original, reconstructed, error_threshold=0.1, radius=5)

    # Display Original Image
    ax = plt.subplot(3, n_images, i + 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original")
    plt.axis("off")
    
    # Display Reconstructed Image
    ax = plt.subplot(3, n_images, i + 1 + n_images)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed")
    plt.axis("off")

    # Display Anomalies Highlighted on Original
    ax = plt.subplot(3, n_images, i + 1 + 2 * n_images)
    plt.imshow(highlighted_image)
    plt.title("Anomalies Highlighted")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    for filename in sorted(os.listdir(folder)):
        # Process only files with the extensions .tif or .gt (if they exist)
        if filename.endswith(".tif") or filename.endswith(".gt"):
            file_path = os.path.join(folder, filename)
            try:
                img = load_img(file_path, target_size=target_size, color_mode="grayscale")
                img = img_to_array(img) / 255.0
                images.append(img)
            except Exception as e:
                print(f"Error loading image {file_path}: {e}")
    return np.array(images)

# Example usage:
test_folder = '/kaggle/input/anomalynds/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test001'
test_images = load_images_from_folder(test_folder)

print(f"Loaded {len(test_images)} images for testing.")


In [ ]:
# Assuming `autoencoder` is your trained model
decoded_imgs = autoencoder.predict(test_images)
print(f"Generated reconstructions for {len(decoded_imgs)} test images.")


In [ ]:
import numpy as np

# Calculate MSE for each test image
reconstruction_errors = np.mean((test_images - decoded_imgs) ** 2, axis=(1, 2, 3))


In [ ]:
anomaly_threshold = np.percentile(reconstruction_errors, 90)
print(f"Anomaly detection threshold (95th percentile): {anomaly_threshold}")


In [ ]:
anomalies = test_images[reconstruction_errors > anomaly_threshold]
normal_images = test_images[reconstruction_errors <= anomaly_threshold]

print(f"Number of anomalies detected: {len(anomalies)}")
print(f"Number of normal images: {len(normal_images)}")


In [ ]:
import matplotlib.pyplot as plt

# Display a few samples of detected anomalies
n_samples = min(len(anomalies), 5)  # Show up to 5 anomalies
plt.figure(figsize=(15, 5))
for i in range(n_samples):
    # Original and reconstructed images of the anomaly
    original = anomalies[i].reshape(128, 128)
    reconstructed = decoded_imgs[reconstruction_errors > anomaly_threshold][i].reshape(128, 128)

    # Show original
    plt.subplot(2, n_samples, i + 1)
    plt.imshow(original, cmap='gray')
    plt.title("Anomalous Original")
    plt.axis("off")

    # Show reconstruction
    plt.subplot(2, n_samples, i + 1 + n_samples)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def highlight_anomalies_on_image(original, reconstructed, error_threshold=0.1, radius=5):
    # Calculate the absolute difference (error map) between the original and reconstructed images
    error_map = np.abs(original - reconstructed)
    
    # Threshold the error map to isolate high-error regions (likely anomalies)
    anomaly_mask = (error_map > error_threshold).astype(np.uint8)
    anomaly_mask = (anomaly_mask * 255).astype(np.uint8)

    # Find contours in the thresholded mask
    contours, _ = cv2.findContours(anomaly_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Convert original image to RGB to draw colored circles
    highlighted_image = cv2.cvtColor((original * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)

    # Draw circles around each contour
    for contour in contours:
        # Calculate the center and radius of each contour
        (x, y), contour_radius = cv2.minEnclosingCircle(contour)
        
        # Draw a circle if the radius meets the minimum size requirement
        if contour_radius > radius:
            center = (int(x), int(y))
            cv2.circle(highlighted_image, center, int(contour_radius), (255, 0, 0), 2)  # Red circle

    return highlighted_image

# Visualize a few anomalies with highlighted regions
n_samples = min(len(anomalies), 5)  # Display up to 5 anomalies
plt.figure(figsize=(15, 8))

for i in range(n_samples):
    # Index of the anomaly
    idx = np.where(reconstruction_errors > anomaly_threshold)[0][i]

    # Original and reconstructed images
    original = test_images[idx].reshape(128, 128)
    reconstructed = decoded_imgs[idx].reshape(128, 128)
    highlighted_image = highlight_anomalies_on_image(original, reconstructed, error_threshold=0.1, radius=5)

    # Plot Original Image
    plt.subplot(3, n_samples, i + 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original Image")
    plt.axis("off")
    
    # Plot Reconstructed Image
    plt.subplot(3, n_samples, i + 1 + n_samples)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed Image")
    plt.axis("off")

    # Plot Anomalies Highlighted on Original Image
    plt.subplot(3, n_samples, i + 1 + 2 * n_samples)
    plt.imshow(highlighted_image)
    plt.title("Anomalies Highlighted")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load all image files with specific extensions from a folder
def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    valid_extensions = (".m", ".tif", ".bmp", ".png", ".jpg", ".jpeg")  # Add other extensions as needed
    for filename in sorted(os.listdir(folder)):
        # Load only files with valid extensions
        if filename.endswith(valid_extensions):
            file_path = os.path.join(folder, filename)
            try:
                img = load_img(file_path, target_size=target_size, color_mode="grayscale")
                img = img_to_array(img) / 255.0
                images.append(img)
            except Exception as e:
                print(f"Skipped file {file_path}: {e}")
    return np.array(images)

# Paths for UCSDped1 and UCSDped2 test datasets
test_path_ped1 = '/kaggle/input/anomalynds/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test'
test_path_ped2 = '/kaggle/input/anomalynds/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Test'

# Load UCSDped1 and UCSDped2 test datasets
test_images_ped1 = []
test_images_ped2 = []

for folder in sorted(os.listdir(test_path_ped1)):
    folder_path = os.path.join(test_path_ped1, folder)
    if os.path.isdir(folder_path):  # Check if the entry is a directory
        test_images_ped1.append(load_images_from_folder(folder_path))
test_images_ped1 = np.concatenate(test_images_ped1)

for folder in sorted(os.listdir(test_path_ped2)):
    folder_path = os.path.join(test_path_ped2, folder)
    if os.path.isdir(folder_path):  # Check if the entry is a directory
        test_images_ped2.append(load_images_from_folder(folder_path))
test_images_ped2 = np.concatenate(test_images_ped2)

print(f"Loaded {len(test_images_ped1)} images from UCSDped1 and {len(test_images_ped2)} images from UCSDped2.")

# Perform predictions
decoded_imgs_ped1 = autoencoder.predict(test_images_ped1)
decoded_imgs_ped2 = autoencoder.predict(test_images_ped2)

# Calculate reconstruction errors
reconstruction_errors_ped1 = np.mean((test_images_ped1 - decoded_imgs_ped1) ** 2, axis=(1, 2, 3))
reconstruction_errors_ped2 = np.mean((test_images_ped2 - decoded_imgs_ped2) ** 2, axis=(1, 2, 3))

# Set anomaly threshold (e.g., 95th percentile)
threshold_ped1 = np.percentile(reconstruction_errors_ped1, 95)
threshold_ped2 = np.percentile(reconstruction_errors_ped2, 95)

anomalies_ped1 = test_images_ped1[reconstruction_errors_ped1 > threshold_ped1]
anomalies_ped2 = test_images_ped2[reconstruction_errors_ped2 > threshold_ped2]

print(f"Number of anomalies detected in UCSDped1: {len(anomalies_ped1)}")
print(f"Number of anomalies detected in UCSDped2: {len(anomalies_ped2)}")

# Function to highlight anomalies in an image
def highlight_anomalies_on_image(original, reconstructed, error_threshold=0.1, radius=5):
    error_map = np.abs(original - reconstructed)
    anomaly_mask = (error_map > error_threshold).astype(np.uint8)
    anomaly_mask = (anomaly_mask * 255).astype(np.uint8)
    contours, _ = cv2.findContours(anomaly_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    highlighted_image = cv2.cvtColor((original * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)
    for contour in contours:
        (x, y), contour_radius = cv2.minEnclosingCircle(contour)
        if contour_radius > radius:
            center = (int(x), int(y))
            cv2.circle(highlighted_image, center, int(contour_radius), (255, 0, 0), 2)  # Red circle
    return highlighted_image

# Visualize anomalies in UCSDped1
n_samples = min(len(anomalies_ped1), 5)  # Show up to 5 anomalies
plt.figure(figsize=(15, 8))
for i in range(n_samples):
    idx = np.where(reconstruction_errors_ped1 > threshold_ped1)[0][i]
    original = test_images_ped1[idx].reshape(128, 128)
    reconstructed = decoded_imgs_ped1[idx].reshape(128, 128)
    highlighted_image = highlight_anomalies_on_image(original, reconstructed, error_threshold=0.1, radius=5)

    # Original Image
    plt.subplot(3, n_samples, i + 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original")
    plt.axis("off")
    
    # Reconstructed Image
    plt.subplot(3, n_samples, i + 1 + n_samples)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed")
    plt.axis("off")

    # Highlighted Anomalies
    plt.subplot(3, n_samples, i + 1 + 2 * n_samples)
    plt.imshow(highlighted_image)
    plt.title("Anomalies Highlighted")
    plt.axis("off")

plt.tight_layout()
plt.show()

# Visualize anomalies in UCSDped2
n_samples = min(len(anomalies_ped2), 5)  # Show up to 5 anomalies
plt.figure(figsize=(15, 8))
for i in range(n_samples):
    idx = np.where(reconstruction_errors_ped2 > threshold_ped2)[0][i]
    original = test_images_ped2[idx].reshape(128, 128)
    reconstructed = decoded_imgs_ped2[idx].reshape(128, 128)
    highlighted_image = highlight_anomalies_on_image(original, reconstructed, error_threshold=0.1, radius=5)

    # Original Image
    plt.subplot(3, n_samples, i + 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original")
    plt.axis("off")
    
    # Reconstructed Image
    plt.subplot(3, n_samples, i + 1 + n_samples)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed")
    plt.axis("off")

    # Highlighted Anomalies
    plt.subplot(3, n_samples, i + 1 + 2 * n_samples)
    plt.imshow(highlighted_image)
    plt.title("Anomalies Highlighted")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import os

def generate_true_labels(test_folder, gt_suffix='_gt', gt_extension='.bmp', valid_extensions=('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
    true_labels = []
    
    # Loop through each subfolder in the test directory
    for root, dirs, files in os.walk(test_folder):
        for folder in sorted(dirs):
            folder_path = os.path.join(root, folder)
            # Check if there's a corresponding *_gt folder
            gt_folder_path = folder_path + gt_suffix
            
            # Collect all image filenames in this folder
            for filename in sorted(os.listdir(folder_path)):
                if filename.lower().endswith(valid_extensions):
                    # Construct the path for the GT file with .bmp extension
                    gt_file_path = os.path.join(gt_folder_path, os.path.splitext(filename)[0] + gt_extension)
                    
                    # Assign anomaly (1) if corresponding .bmp file exists in *_gt folder, else normal (0)
                    if os.path.exists(gt_file_path):
                        true_labels.append(1)
                    else:
                        true_labels.append(0)
    
    return true_labels

# Define paths to UCSDped1 and UCSDped2 test folders
test_folder_ped1 = "/kaggle/input/anomalynds/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test"
test_folder_ped2 = "/kaggle/input/anomalynds/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Test"

# Generate true labels for both test sets
true_labels_ped1 = generate_true_labels(test_folder_ped1)
true_labels_ped2 = generate_true_labels(test_folder_ped2)

# Concatenate labels if you want to combine both datasets
true_labels = true_labels_ped1 + true_labels_ped2

print("True Labels for Test Images:", true_labels)


In [ ]:
# Checking the Lengths of true_labels, predicted_labels, and reconstruction_errors After Truncating true_labels

print(f"Length of true_labels: {len(true_labels)}")
print(f"Length of predicted_labels: {len(predicted_labels)}")
print(f"Length of reconstruction_errors: {len(reconstruction_errors)}")


In [ ]:
# Truncating true_labels to the first 200 elements

true_labels = true_labels[:200]


In [ ]:
# Checking the Lengths of true_labels, predicted_labels, and reconstruction_errors

print(f"Adjusted Length of true_labels: {len(true_labels)}")
print(f"Length of predicted_labels: {len(predicted_labels)}")
print(f"Length of reconstruction_errors: {len(reconstruction_errors)}")


In [ ]:
# Performance Metrics Calculation for Anomaly Detection (Accuracy, Precision, Recall, F1 Score)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
# Distribution of Reconstruction Errors with 50th Percentile Anomaly Threshold

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set a lower threshold (e.g., 90th percentile) to increase sensitivity
threshold = np.percentile(reconstruction_errors, 90)
predicted_labels = (reconstruction_errors > threshold).astype(int)

# Calculate performance metrics with zero_division to handle cases where there are no positives
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, zero_division=1)
recall = recall_score(true_labels, predicted_labels, zero_division=1)
f1 = f1_score(true_labels, predicted_labels, zero_division=1)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
# Setting Anomaly Threshold at 50th Percentile of Reconstruction Errors

import matplotlib.pyplot as plt

plt.hist(reconstruction_errors, bins=50)
plt.axvline(x=threshold, color='r', linestyle='--', label='Anomaly Threshold')
plt.xlabel("Reconstruction Error")
plt.ylabel("Frequency")
plt.title("Distribution of Reconstruction Errors")
plt.legend()
plt.show()


In [ ]:
threshold = np.percentile(reconstruction_errors, 50)  # Set a lower percentile


In [ ]:
# Distribution of Reconstruction Errors with Anomaly Threshold

import matplotlib.pyplot as plt

plt.hist(reconstruction_errors, bins=50)
plt.axvline(x=threshold, color='r', linestyle='--', label='Anomaly Threshold')
plt.xlabel("Reconstruction Error")
plt.ylabel("Frequency")
plt.title("Distribution of Reconstruction Errors")
plt.legend()
plt.show()


In [ ]:
# Predicted Labels for Anomalies Based on Reconstruction Errors
predicted_labels = (reconstruction_errors > threshold).astype(int)


In [ ]:
#Binary Cross-Entropy (BCE) Anomaly Detection for Image Reconstruction
import tensorflow as tf
import numpy as np

# Define Binary Cross-Entropy Loss
bce = tf.keras.losses.BinaryCrossentropy()

# Calculate BCE for each image in the test set
bce_errors = np.array([bce(test_images[i], decoded_imgs[i]).numpy() for i in range(len(test_images))])

# Set a threshold for anomalies (e.g., 95th percentile)
anomaly_threshold_bce = np.percentile(bce_errors, 95)
anomalies_bce = test_images[bce_errors > anomaly_threshold_bce]

print(f"Average BCE for anomalous images: {np.mean(bce_errors[bce_errors > anomaly_threshold_bce])}")
print(f"Number of anomalies detected based on BCE: {len(anomalies_bce)}")

# Display threshold for reference
print(f"BCE Anomaly Threshold: {anomaly_threshold_bce}")


In [ ]:
#Visualization of Top 10 Anomalies Detected: Original, Reconstructed, and Difference Images
import matplotlib.pyplot as plt

# Get indices of the anomalies based on BCE errors
anomaly_indices = np.where(bce_errors > anomaly_threshold_bce)[0]

# Limit to the top 10 anomalies
top_anomaly_indices = anomaly_indices[:10]

plt.figure(figsize=(15, 6))
for i, idx in enumerate(top_anomaly_indices):
    # Original image
    ax = plt.subplot(3, 10, i + 1)
    plt.imshow(test_images[idx].reshape(128, 128), cmap='gray')
    plt.title("Original")
    plt.axis("off")
    
    # Reconstructed image
    ax = plt.subplot(3, 10, i + 11)
    plt.imshow(decoded_imgs[idx].reshape(128, 128), cmap='gray')
    plt.title("Reconstructed")
    plt.axis("off")
    
    # Difference (Highlight Anomalies)
    diff = np.abs(test_images[idx] - decoded_imgs[idx])
    ax = plt.subplot(3, 10, i + 21)
    plt.imshow(diff.reshape(128, 128), cmap='hot')
    plt.title("Difference")
    plt.axis("off")

plt.suptitle("Top 10 Anomalies Detected")
plt.show()
